In [ ]:
# Import necessary libraries
%pip install --upgrade -q aiobotocore
%pip install -q  xgboost==1.3.1
!pip install PyAthena[SQLAlchemy]

In [ ]:
import pandas as pd
import sagemaker
import json
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import boto3

In [ ]:
# Set SageMaker and S3 client variables
sess = sagemaker.Session()
region = sess.boto_region_name
sagemaker_role = sagemaker.get_execution_role()

# Set write S3 bucket and location
write_bucket = "appl-financial-csv"
write_prefix = "model-output"
model_key = f"{write_prefix}/model"
output_key = f"{write_prefix}/output"

# Create an S3 client
s3_client = boto3.client("s3")

In [ ]:
# Connect to Athena
s3_staging_dir = "s3://appl-financial-csv/raw"
connection_string = f"awsathena+rest://:@athena.us-west-1.amazonaws.com:443/appl-db-financial?s3_staging_dir={s3_staging_dir}"
engine = create_engine(connection_string)
data = pd.read_sql('SELECT * FROM "appl-db-financial"."raw_data_output";', engine)

In [ ]:
# Initialize metrics_data
metrics_data = {
    "regression_metrics": {
        "validation:mae": {
            "value": [],
            "standard_deviation": []
        },
        "train:mae": {
            "value": [],
            "standard_deviation": []
        },
    }
}

In [ ]:
# Initialize model parameters
hyperparams = {
    "max_depth": 3,
    "eta": 0.2,
    "objective": "binary:logistic",
    "subsample" : 0.8,
    "colsample_bytree" : 0.8,
    "min_child_weight" : 3
}

num_boost_round = 100
early_stopping_rounds = 10

In [ ]:
# Data Preprocessing
  # Convert type
data['Datetime'] = pd.to_datetime(data['Datetime'])

  # Sort the data by timestamp
data = data.sort_values(by='Datetime')
data_withDateTime = data.sort_values(by='Datetime')
data = data.drop('Datetime',axis=1)

  # Remove 'Q' from the quarter column
data['Period'] = pd.to_numeric(data['Period'].str.replace('Q', ''))

  # Encoding 'Symbol' because categorical column
data = pd.concat([data, pd.get_dummies(data['Symbol'], prefix='symbol')], axis=1)
data = data.drop('Symbol', axis=1)  # Drop the original column

  # Separate features and labels for splitting
label_col = "Close"
features = data.drop(label_col, axis=1)
labels = data[label_col]

In [ ]:
# Perform TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

for split_idx, (train_index, test_index) in enumerate(tscv.split(features), 1):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

    # Convert to XGBoost DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Train XGBoost model
    xgb_model = xgb.train(
        params=hyperparams,
        dtrain=dtrain,
        num_boost_round=num_boost_round,
        early_stopping_rounds=early_stopping_rounds,
        evals=[(dtest, "validation")],
        verbose_eval=10
    )

    # Evaluate the model
    predictions = xgb_model.predict(dtest)
    mae = mean_absolute_error(y_test, predictions)

    # Update metrics_data
    metrics_data["regression_metrics"]["validation:mae"]["value"].append(float(mae))
    metrics_data["regression_metrics"]["validation:mae"]["standard_deviation"].append(float(mae.std()))

    # Create a DataFrame with actual and predicted Close values
    results = pd.DataFrame({'Actual': data['Close'].iloc[test_index], 'Predicted': predictions, 'Datetime': data_withDateTime['Datetime'].iloc[test_index]})

    # Time series plot
    plt.figure(figsize=(10, 6))
    plt.plot(results['Datetime'], results['Actual'], label='Actual', marker='o')
    plt.plot(results['Datetime'], results['Predicted'], label='Predicted', marker='o')
    plt.title(f'Time Series Plot - Split {split_idx}')
    plt.xlabel('Datetime')
    plt.ylabel('Close')
    plt.legend()
    plt.show()

# Set mean and standard deviation for the metrics
metrics_data["regression_metrics"]["validation:mae"]["value"] = round(metrics_data["regression_metrics"]["validation:mae"]["value"], 2)
metrics_data["regression_metrics"]["validation:mae"]["standard_deviation"] = round(metrics_data["regression_metrics"]["validation:mae"]["standard_deviation"], 2)

print(f"Cross-validated train-mae: {metrics_data['regression_metrics']['train:mae']['value']}")
print(f"Cross-validated validation-mae: {metrics_data['regression_metrics']['validation:mae']['value']} +/- {metrics_data['regression_metrics']['validation:mae']['standard_deviation']}")

In [ ]:
# Upload model and performance metrics to S3
metrics_location = f"{output_key}/metrics.json"
model_location = f"{model_key}/xgboost-model"
# Convert xgb_model to a binary buffer for upload
model_buffer = xgb_model.save_raw()
# Upload metrics.json
s3_client.put_object(Body=json.dumps(metrics_data), Bucket=write_bucket, Key=metrics_location)
# Upload xgboost-model
s3_client.put_object(Body=model_buffer, Bucket=write_bucket, Key=model_location)

In [ ]:
# Combine features and labels for the entire dataset
dall = xgb.DMatrix(features, label=labels, enable_categorical=True)
# Train XGBoost model on the entire dataset
xgb_model_full = xgb.train(
    params=hyperparams,
    dtrain=dall,
    num_boost_round=num_boost_round,
    early_stopping_rounds=early_stopping_rounds,
    evals=[(dall, "validation")],
    verbose_eval=10
)
# Evaluate the model on the entire dataset
predictions_full = xgb_model_full.predict(dall)
mae_full = mean_absolute_error(labels, predictions_full)

# Set mean and standard deviation for the metrics for the entire dataset
metrics_data["regression_metrics"]["validation:mae"]["value"] = round(metrics_data["regression_metrics"]["validation:mae"]["value"], 2)
metrics_data["regression_metrics"]["validation:mae"]["standard_deviation"] = round(metrics_data["regression_metrics"]["validation:mae"]["standard_deviation"], 2)

# Create a DataFrame with actual and predicted Close values for the entire dataset
results_full = pd.DataFrame({'Actual': labels, 'Predicted': predictions_full})

plt.figure(figsize=(12, 6))
plt.plot(data_withDateTime['Datetime'], results_full['Actual'], label='Actual', marker='o')
plt.plot(data_withDateTime['Datetime'], results_full['Predicted'], label='Predicted', marker='o')
plt.title('Actual vs. Predicted Close Over Time')
plt.xlabel('Datetime')
plt.ylabel('')
plt.legend()
plt.show()

In [ ]:
###Predicting future dates locally
from datetime import datetime, timedelta

last_date = data_withDateTime['Datetime'].max()
# Create a new datapoint for the next day
future_date = last_date + timedelta(days=1)
# Use the last row of dataset as the new datapoint
last_data_point = data.tail(1).copy()
last_data_point['Datetime'] = future_date
# Drop the target variable ('Close') if it's present in the features
if 'Close' in last_data_point.columns:
    last_data_point = last_data_point.drop('Close', axis=1)
    last_data_point = last_data_point.drop('Datetime', axis=1)

# Convert the new datapoint to XGBoost DMatrix
dnew = xgb.DMatrix(last_data_point, enable_categorical=True)

# Make predictions for the new datapoint
prediction_for_new_datapoint = xgb_model_full.predict(dnew)[0]
print(f'Prediction for the new datapoint on {future_date}: {prediction_for_new_datapoint:.4f}')

# Create a DataFrame with the new datapoint and prediction
new_datapoint = pd.DataFrame({'Actual': [None], 'Predicted': [prediction_for_new_datapoint], 'Datetime': [future_date]})

results_last_500 = results_full.iloc[-200:]
data_withDateTime_last500 = data_withDateTime.iloc[-200:]

plt.figure(figsize=(12, 6))
plt.plot(data_withDateTime_last500['Datetime'], results_last_500['Actual'], label='Actual', marker='o')
plt.plot(data_withDateTime_last500['Datetime'], results_last_500['Predicted'], label='Predicted', marker='o')
plt.scatter(future_date, prediction_for_new_datapoint, color='red', marker='x', label='Next Date')
plt.title('Actual vs. Predicted Close Over Time')
plt.xlabel('Datetime')
plt.ylabel('Close')
plt.legend()
plt.show()

# Print MAE
print(f'Mean Absolute Error (MAE): {mae:.4f}')